Algoritmos

In [1]:
import math
from heapq import heappush, heappop


class Graph:
    def __init__(self):
        self.vertexMap = dict()

    def addVertex(self, v):
        self.vertexMap[v] = dict()

    def removeVertex(self, v):
        if v in self.vertexMap:
            for i, j in self.vertexMap[v].copy():
                print(f"e->{(i,j)}")
                self.removeEdge(i, j)
            del self.vertexMap[v]

    def vertices(self):
        return list(self.vertexMap.keys())

    def adjacents(self, v):
        return [j for (i, j) in self.outgoing(v)]

    def addEdge(self, u, v, data):
        if (u in self.vertexMap) and (v in self.vertexMap):
            self.vertexMap[u][(u, v)] = data
            self.vertexMap[v][(v, u)] = data
        else:
            raise ValueError(
                f"One or both of the V {u} and {v} are not present in the Graph!"
            )

    def removeEdge(self, u, v):
        if ((u, v) in self.vertexMap[u]) and ((v, u) in self.vertexMap[v]):
            del self.vertexMap[u][(u, v)]
            del self.vertexMap[v][(v, u)]

    def edges(self):
        ret = []
        for e in self.vertexMap.values():
            if len(e.keys()):
                ret.extend(list(e.keys()))
        return ret

    def getEdge(self, u, v):
        return self.vertexMap[u][(u, v)]

    def outgoing(self, v):
        return list(self.vertexMap[v].keys())

    def outdegree(self, v):
        return len(self.vertexMap[v])

    def incoming(self, v):
        return [(j, i) for (i, j) in self.vertexMap[v]]

    def indegree(self, v):
        return len(self.vertexMap[v])

    def path(self, v):
        ret = ""
        visited = set()
        visited.add(v)
        stack = []
        stack.append((v, None))
        while stack:
            (v, p) = stack.pop()
            if p:
                ret += f"{p}--{self.getEdge(p,v)}--{v}  "

            for u in self.adjacents(v):
                if u not in visited:
                    visited.add(u)
                    stack.append((u, v))

        return ret.strip()


def get_min_edge(G: Graph) -> tuple[str, str, int]:
    edges = G.edges()
    first_edge = edges[0]
    min_edge = (first_edge[0], first_edge[1], G.getEdge(first_edge[0], first_edge[1]))
    for u, v in edges[1:]:
        current_weight = G.getEdge(u, v)
        if current_weight < min_edge[2]:
            min_edge = (u, v, current_weight)
    return min_edge


def get_adjacent_edges(G: Graph, v: list[str]):
    edges = []
    for vertice in v:
        adjacents = G.adjacents(vertice)
        for a in adjacents:
            weight = G.getEdge(vertice, a)
            edges.append((vertice, a, weight))
    return edges


def get_edges_with_weight(G: Graph) -> list[tuple[str, str, int]]:
    edges = []
    g_edges = G.edges()
    for u, v in g_edges:
        weight = G.getEdge(u, v)
        edges.append((u, v, weight))
    return edges


def dfs_recursive(G: Graph, v: str, visited: set, path: list):

    visited.add(v)

    for a in G.adjacents(v):
        if a not in visited:
            path.append(a)
            dfs_recursive(G, a, visited, path)


def dfs(G: Graph, v: str):
    visited = set()
    path = []

    dfs_recursive(G, v, visited, path)
    return path


def prims(G: Graph):
    T = Graph()
    min_edge = get_min_edge(G)

    T.addVertex(min_edge[0])
    T.addVertex(min_edge[1])
    T.addEdge(*min_edge)

    while (len(T.edges()) // 2) < len(G.vertices()) - 1:
        vertices = T.vertices()
        edges = get_adjacent_edges(G, vertices)
        sorted_edges = sorted(edges, key=lambda e: e[2])
        for u, v, weight in sorted_edges:
            if u not in vertices:
                T.addVertex(u)
                T.addEdge(u, v, weight)
                break
            elif v not in vertices:
                T.addVertex(v)
                T.addEdge(u, v, weight)
                break
    return T


def kruskals(G: Graph):
    T = Graph()
    edges = get_edges_with_weight(G)
    edges = sorted(edges, key=lambda e: e[2])

    while (len(T.edges()) // 2) < len(G.vertices()) - 1:
        for u, v, weight in edges:
            has_edge = (u, v) in T.edges()
            try:
                has_cicle = v in dfs(T, u)
            except KeyError:
                has_cicle = False
            if not has_edge and not has_cicle:
                if u not in T.vertices():
                    T.addVertex(u)
                if v not in T.vertices():
                    T.addVertex(v)
                T.addEdge(u, v, weight)
                break

    return T

In [2]:
# Graph 1
g = Graph()
g.addVertex("a")
g.addVertex("b")
g.addVertex("c")
g.addVertex("d")
g.addVertex("e")
g.addVertex("f")
g.addVertex("g")
g.addEdge("a", "b", 28)
g.addEdge("a", "c", 10)
g.addEdge("c", "f", 25)
g.addEdge("b", "d", 14)
g.addEdge("b", "e", 16)
g.addEdge("d", "f", 24)
g.addEdge("d", "g", 18)
g.addEdge("f", "g", 22)
g.addEdge("g", "e", 12)

p = prims(g)
assert sorted(p.vertices()) == ["a", "b", "c", "d", "e", "f", "g"]
assert sorted(p.edges()) == sorted(
    [
        ("a", "c"),
        ("b", "e"),
        ("b", "d"),
        ("c", "a"),
        ("c", "f"),
        ("d", "b"),
        ("e", "g"),
        ("e", "b"),
        ("f", "c"),
        ("f", "g"),
        ("g", "f"),
        ("g", "e"),
    ]
)
assert p.path("a") == "a--10--c  c--25--f  f--22--g  g--12--e  e--16--b  b--14--d"

p = kruskals(g)
assert sorted(p.vertices()) == ["a", "b", "c", "d", "e", "f", "g"]
assert sorted(p.edges()) == sorted(
    [
        ("a", "c"),
        ("b", "e"),
        ("b", "d"),
        ("c", "a"),
        ("c", "f"),
        ("d", "b"),
        ("e", "g"),
        ("e", "b"),
        ("f", "c"),
        ("f", "g"),
        ("g", "f"),
        ("g", "e"),
    ]
)
assert p.path("a") == "a--10--c  c--25--f  f--22--g  g--12--e  e--16--b  b--14--d"

# Graph 2
g = Graph()
g.addVertex("a")
g.addVertex("b")
g.addVertex("c")
g.addVertex("d")
g.addVertex("e")
g.addVertex("f")
g.addVertex("g")
g.addVertex("h")
g.addEdge("a", "b", 6)
g.addEdge("a", "c", 4)
g.addEdge("b", "c", 5)
g.addEdge("b", "e", 14)
g.addEdge("b", "h", 10)
g.addEdge("c", "f", 2)
g.addEdge("c", "d", 9)
g.addEdge("e", "h", 3)
g.addEdge("f", "g", 15)
g.addEdge("f", "h", 8)

p = prims(g)
assert sorted(p.vertices()) == ["a", "b", "c", "d", "e", "f", "g", "h"]
assert sorted(p.edges()) == sorted(
    [
        ("a", "c"),
        ("b", "c"),
        ("c", "f"),
        ("c", "a"),
        ("c", "b"),
        ("c", "d"),
        ("d", "c"),
        ("e", "h"),
        ("f", "c"),
        ("f", "h"),
        ("f", "g"),
        ("g", "f"),
        ("h", "f"),
        ("h", "e"),
    ]
)
assert p.path("a") == "a--4--c  c--9--d  c--5--b  c--2--f  f--15--g  f--8--h  h--3--e"

p = kruskals(g)
assert sorted(p.vertices()) == ["a", "b", "c", "d", "e", "f", "g", "h"]
assert sorted(p.edges()) == sorted(
    [
        ("a", "c"),
        ("b", "c"),
        ("c", "f"),
        ("c", "a"),
        ("c", "b"),
        ("c", "d"),
        ("d", "c"),
        ("e", "h"),
        ("f", "c"),
        ("f", "h"),
        ("f", "g"),
        ("g", "f"),
        ("h", "f"),
        ("h", "e"),
    ]
)
assert p.path("a") == "a--4--c  c--9--d  c--5--b  c--2--f  f--15--g  f--8--h  h--3--e"

print("Here comes the sun...")

Here comes the sun...
